## Read results from exported files - after deep learning training and testing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

import seaborn as sns
import matplotlib
import matplotlib.animation as animation

#from datetime import timedelta
from hhpy.plotting import animplot

import datetime
from pandas import datetime as dt
from matplotlib import pyplot

import os, fnmatch

from sklearn import preprocessing
from sklearn.model_selection import train_test_split


from utils.input_data import read_data_sets
import utils.datasets as ds
import utils.augmentation as aug
import utils.helper as hlp

import itertools
import pickle


# List of groups of columns for EEG, ECG and ET - They are used to run experiments for combination between groups
eeg_current_columns1 = ['EEG_Decon_Poz', 'EEG_Decon_P3','EEG_Decon_P4']
eeg_current_columns2 = ['EEG_Decon_Cz', 'EEG_Decon_C3', 'EEG_Decon_C4']
eeg_current_columns3 = ['EEG_Decon_Fz', 'EEG_Decon_F3', 'EEG_Decon_F4']

ecg_columns1 = ['Other_Vsensebatt_RAW_Shim_Exg_02', 'Other_EXG1_Status_RAW_Shim_Exg_02', 'Other_EXG2_Status_RAW_Shim_Exg_02', 'Other_ECG_LL-RA_RAW_Shim_Exg_02', 'Other_ECG_LA-RA_RAW_Shim_Exg_02', 'Other_ECG_Vx-RL_RAW_Shim_Exg_02']
ecg_columns2 = ['Other_Vsensebatt_CAL_Shim_Exg_02', 'Other_ECG_LL-RA_CAL_Shim_Exg_02', 'Other_ECG_LA-RA_CAL_Shim_Exg_02', 'Other_ECG_Vx-RL_CAL_Shim_Exg_02']
ecg_columns3 = ['Other_Heart_Rate_ECG_LL-RA_ALG_Shim_Exg_02', 'Other_IBI_ECG_LL-RA_ALG_Shim_Exg_02']


#et_columns1 = []
et_columns1 = ['ET_Gaze_2D_ET_Gazeleftx', 'ET_Gaze_2D_ET_Gazelefty', 'ET_Gaze_2D_ET_Gazerightx', 'ET_Gaze_2D_ET_Gazerighty', 'ET_Capture_ET_Cameraleftx', 'ET_Capture_ET_Cameralefty', 'ET_Capture_ET_Camerarightx', 'ET_Capture_ET_Camerarighty', 'ET_Head_ET_Headrotationx', 'ET_Head_ET_Headrotationy', 'ET_Head_ET_Headrotationz', 'ET_Head_ET_Headpositionvectorx', 'ET_Head_ET_Headpositionvectory', 'ET_Head_ET_Headpositionvectorz','ET_Head_ET_Headvelocityx', 'ET_Head_ET_Headvelocityy', 'ET_Head_ET_Headvelocityz','ET_Head_ET_Headangularvelocityx', 'ET_Head_ET_Headangularvelocityy','ET_Head_ET_Headangularvelocityz', 'ET_Other_ET_Timesignal', 'ET_Distance_ET_Distanceleft','ET_Distance_ET_Distanceright']
et_columns2 = ['ET_Pupil_ET_Pupilleft', 'ET_Expression_ET_Lefteyeopenness''ET_Expression_ET_Lefteyesqueeze','ET_Expression_ET_Lefteyefrown']
et_columns3 = ['ET_Pupil_ET_Pupilright', 'ET_Expression_ET_Righteyesqueeze', 'ET_Expression_ET_Righteyeopenness', 'ET_Expression_ET_Righteyefrown']
              

#all_columns_groups = {'eeg_current_columns1':eeg_current_columns1, 'eeg_current_columns2':eeg_current_columns2, 'eeg_current_columns3':eeg_current_columns3, 'ecg_columns1':ecg_columns1, 'ecg_columns2':ecg_columns2, 'ecg_columns3':ecg_columns3, 'et_columns1':et_columns1, 'et_columns2':et_columns2, 'et_columns3':et_columns3}


all_columns = {'B-Alert Decontaminated EEG' : {'eeg_current_columns1':eeg_current_columns1, 'eeg_current_columns2':eeg_current_columns2, 'eeg_current_columns3':eeg_current_columns3},
               'Eyetracker HTC VIVE Pro Eye' : {'et_columns1':et_columns1, 'et_columns2':et_columns2, 'et_columns3':et_columns3},
                'Shimmer shim exg 02 5F2F ECG': { 'ecg_columns1':ecg_columns1, 'ecg_columns2':ecg_columns2, 'ecg_columns3':ecg_columns3}
              }

groups = []
for i in all_columns:
    groups = groups + list(all_columns[i].keys())

# return the name of the list of groups
def get_key(val):
    for key, value in all_columns.items():
        if val in value:
            return key
 
    return "key doesn't exist"
 

groupstoprint = []
for i in all_columns:
    #print(all_columns[i].keys())
    #print(i)
    for j in all_columns[i]:
        print(j, end=',')
    for k in all_columns[i]:
        for f in all_columns[i][k]:
            print(f)
    groupstoprint = groupstoprint + list(all_columns[i].keys())


In /home/deep01/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/deep01/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/deep01/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/deep01/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/deep01/.local/lib/python3.6/site-packages/matplotlib/mpl-data/s

eeg_current_columns1,eeg_current_columns2,eeg_current_columns3,EEG_Decon_Poz
EEG_Decon_P3
EEG_Decon_P4
EEG_Decon_Cz
EEG_Decon_C3
EEG_Decon_C4
EEG_Decon_Fz
EEG_Decon_F3
EEG_Decon_F4
et_columns1,et_columns2,et_columns3,ET_Gaze_2D_ET_Gazeleftx
ET_Gaze_2D_ET_Gazelefty
ET_Gaze_2D_ET_Gazerightx
ET_Gaze_2D_ET_Gazerighty
ET_Capture_ET_Cameraleftx
ET_Capture_ET_Cameralefty
ET_Capture_ET_Camerarightx
ET_Capture_ET_Camerarighty
ET_Head_ET_Headrotationx
ET_Head_ET_Headrotationy
ET_Head_ET_Headrotationz
ET_Head_ET_Headpositionvectorx
ET_Head_ET_Headpositionvectory
ET_Head_ET_Headpositionvectorz
ET_Head_ET_Headvelocityx
ET_Head_ET_Headvelocityy
ET_Head_ET_Headvelocityz
ET_Head_ET_Headangularvelocityx
ET_Head_ET_Headangularvelocityy
ET_Head_ET_Headangularvelocityz
ET_Other_ET_Timesignal
ET_Distance_ET_Distanceleft
ET_Distance_ET_Distanceright
ET_Pupil_ET_Pupilleft
ET_Expression_ET_LefteyeopennessET_Expression_ET_Lefteyesqueeze
ET_Expression_ET_Lefteyefrown
ET_Pupil_ET_Pupilright
ET_Expression_ET_Righ

In [2]:
def printAllResTRAINTestWithFolds(filePrefix,  all_data_by_event_combination_train, all_data_by_event_combination_test, nb_fold=5):
    AlleventsCombination = ['B-Alert Decontaminated EEG', 'B-Alert EEG',   'Eyetracker HTC VIVE Pro Eye','R Analysis GazeAnalysis I-VT filter','Shimmer GSR 4B59', 'Shimmer shim exg 02 5F2F ECG']
    i = 0    
    print("id,Model#,train data #, test data #,Featurs #", end=",")

    for item in AlleventsCombination:
        print(item, end=",")
    #print("Accuracy")
    for foldId in range(nb_fold):
        #if foldId == nb_fold-1:
        #    print("fold"+str(foldId))
        #else:
        print("fold"+str(foldId), end=",")

    print("accuracy")

    all_data_by_event_combinationValues_train = list(all_data_by_event_combination_train.values()) #[0]
    all_data_by_event_combinationValues_test = list(all_data_by_event_combination_test.values()) #[0]

    for L in range(1,len(AlleventsCombination)+1):
        for subset, key in zip(itertools.combinations(AlleventsCombination, L), all_data_by_event_combination_train.keys()):
            eventsCombination = list(subset)
            #print(eventsCombination, " : ", key)
            print(str(i)+",Model "+str(i)+","+str(np.shape(all_data_by_event_combinationValues_train[i])[0])+","+str(np.shape(all_data_by_event_combinationValues_test[i])[0])+","+str(np.shape(all_data_by_event_combinationValues_train[i])[1]), end=",")
            for item in AlleventsCombination:
                print(item in eventsCombination, end =",")
            #with open('/home/deep01/zaher/MLTMoutput_2C/MLTM_zaher_model_'+str(i)+'.txt', 'r') as f:
            #with open('/home/deep01/zaher/MLTMoutput_2C/MLTM_zaher_model_'+str(i)+'.txt', 'r') as f:
            
            accuracyAverage = 0
            for foldId in range(nb_fold):
                #filePrefix = "fold"+str(foldId)+"_MLTM_zaher_2C_no_aug_by_post_";
                with open('/home/deep01/zaher/MLTMoutput/byfolds/fold'+str(foldId)+'_'+filePrefix+str(i)+'.txt', 'r') as f:
                    lines = f.read().splitlines()
                    last_line = lines[-1]
                    #print (last_line)
                    accuracy = float(last_line[18:])
                    accuracyAverage = accuracyAverage + accuracy
                    print(accuracy, end=",")
                    if foldId == nb_fold-1:
                        #print(last_line[18:], end="," )
                        print(accuracyAverage/nb_fold)
                    #else:
                        #print(last_line[18:], end=",")
            i +=1

            

def printAllResTRAINTestFeatures(filePrefix, all_data_by_event_combination_train, all_data_by_event_combination_test):
    #AlleventsCombination = ['B-Alert Decontaminated EEG', 'B-Alert EEG',   'Eyetracker HTC VIVE Pro Eye','R Analysis GazeAnalysis I-VT filter','Shimmer GSR 4B59', 'Shimmer shim exg 02 5F2F ECG']
    i = 0
    print("id,Model#,train data #,test data #,Featurs #", end=",")

    for item in groups:
        print(item, end=",")
    print("Accuracy")


    all_data_by_event_combinationValues_train = list(all_data_by_event_combination_train.values()) #[0]
    all_data_by_event_combinationValues_test = list(all_data_by_event_combination_test.values()) #[0]

    for L in range(2,len(groups)+1):
        for subset, key in zip(itertools.combinations(groups, L), all_data_by_event_combination_train.keys()):
            eventsCombination = list(subset)
            #print(eventsCombination, " : ", key)
            print(str(i)+",Model "+str(i)+","+str(np.shape(all_data_by_event_combinationValues_train[i])[0])+","+str(np.shape(all_data_by_event_combinationValues_test[i])[0])+","+str(np.shape(all_data_by_event_combinationValues_train[i])[1]), end=",")
            for item in groups:
                print(item in eventsCombination, end =",")
            #with open('/home/deep01/zaher/MLTMoutput_2C/MLTM_zaher_model_'+str(i)+'.txt', 'r') as f:
            #with open('/home/deep01/zaher/MLTMoutput_2C/MLTM_zaher_model_'+str(i)+'.txt', 'r') as f:
            with open('/home/deep01/zaher/MLTMoutput/'+filePrefix+str(i)+'.txt', 'r') as f:
                lines = f.read().splitlines()
                last_line = lines[-1]
                #print (last_line)
                print(last_line[18:])

            i +=1
            
def printAllResTRAINTestFeaturesWithFolds(filePrefix, all_data_by_event_combination_train, all_data_by_event_combination_test, nb_fold=5):
    i = 0    
    print("id,Model#,train data #,test data #,Featurs #", end=",")

    for item in groups:
        print(item, end=",")

    for foldId in range(nb_fold):
        #if foldId == nb_fold-1:
        #    print("fold"+str(foldId))
        #else:
        print("fold"+str(foldId), end=",")

    print("accuracy")

    all_data_by_event_combinationValues_train = list(all_data_by_event_combination_train.values()) #[0]
    all_data_by_event_combinationValues_test = list(all_data_by_event_combination_test.values()) #[0]

    for L in range(2,len(groups)+1):
        for subset, key in zip(itertools.combinations(groups, L), all_data_by_event_combination_train.keys()):
            eventsCombination = list(subset)
            #print(eventsCombination, " : ", key)
            print(str(i)+",Model "+str(i)+","+str(np.shape(all_data_by_event_combinationValues_train[i])[0])+","+str(np.shape(all_data_by_event_combinationValues_test[i])[0])+","+str(np.shape(all_data_by_event_combinationValues_train[i])[1]), end=",")
            for item in groups:
                print(item in eventsCombination, end =",")
            
            accuracyAverage = 0
            for foldId in range(nb_fold):
                #filePrefix = "fold"+str(foldId)+"_MLTM_zaher_2C_no_aug_by_post_";
                with open('/home/deep01/zaher/MLTMoutput/byfolds/fold'+str(foldId)+'_'+filePrefix+str(i)+'.txt', 'r') as f:
                    lines = f.read().splitlines()
                    last_line = lines[-1]
                    #print (last_line)
                    accuracy = float(last_line[18:])
                    accuracyAverage = accuracyAverage + accuracy
                    print(accuracy, end=",")
                    if foldId == nb_fold-1:
                        #print(last_line[18:], end="," )
                        print(accuracyAverage/nb_fold)
                    #else:
                        #print(last_line[18:], end=",")
            i +=1

def printAllResTRAINTest(filePrefix, all_data_by_event_combination):
    AlleventsCombination = ['B-Alert Decontaminated EEG', 'B-Alert EEG',   'Eyetracker HTC VIVE Pro Eye','R Analysis GazeAnalysis I-VT filter','Shimmer GSR 4B59', 'Shimmer shim exg 02 5F2F ECG']
    i = 0
    print("id,Model#,data #,Featurs #", end=",")

    for item in AlleventsCombination:
        print(item, end=",")
    print("Accuracy")


    all_data_by_event_combinationValues = list(all_data_by_event_combination.values()) #[0]

    for L in range(1,len(AlleventsCombination)+1):
        for subset, key in zip(itertools.combinations(AlleventsCombination, L), all_data_by_event_combination.keys()):
            eventsCombination = list(subset)
            #print(eventsCombination, " : ", key)
            print(str(i)+",Model "+str(i)+","+str(np.shape(all_data_by_event_combinationValues[i])[0])+","+str(np.shape(all_data_by_event_combinationValues_test[i])[1]), end=",")
            for item in AlleventsCombination:
                print(item in eventsCombination, end =",")
            #with open('/home/deep01/zaher/MLTMoutput_2C/MLTM_zaher_model_'+str(i)+'.txt', 'r') as f:
            #with open('/home/deep01/zaher/MLTMoutput_2C/MLTM_zaher_model_'+str(i)+'.txt', 'r') as f:
            with open('/home/deep01/zaher/MLTMoutput/'+filePrefix+str(i)+'.txt', 'r') as f:
                lines = f.read().splitlines()
                last_line = lines[-1]
                #print (last_line)
                print(last_line[18:])

            i +=1
            

def printAllResWithoutShape(filePrefix):
    AlleventsCombination = ['B-Alert Decontaminated EEG', 'B-Alert EEG',   'Eyetracker HTC VIVE Pro Eye','R Analysis GazeAnalysis I-VT filter','Shimmer GSR 4B59', 'Shimmer shim exg 02 5F2F ECG']
    i = 0
    print("id,Model#", end=",")

    for item in AlleventsCombination:
        print(item, end=",")
    print("Accuracy")



    for L in range(1,len(AlleventsCombination)+1):
        for subset in itertools.combinations(AlleventsCombination, L):
            eventsCombination = list(subset)
            #print(eventsCombination, " : ", key)
            print(str(i)+",Model "+str(i), end=",")
            for item in AlleventsCombination:
                print(item in eventsCombination, end =",")
            #with open('/home/deep01/zaher/MLTMoutput_2C/MLTM_zaher_model_'+str(i)+'.txt', 'r') as f:
            #with open('/home/deep01/zaher/MLTMoutput_2C/MLTM_zaher_model_'+str(i)+'.txt', 'r') as f:
            with open('/home/deep01/zaher/MLTMoutput/'+str(filePrefix)+str(i)+'.txt', 'r') as f:
                lines = f.read().splitlines()
                last_line = lines[-1]
                #print (last_line)
                if 'Final Accuracy' in last_line:
                    print(last_line[18:])
                else:
                    print(0)

            i +=1

def printAllResWithoutShapeForAllUsers(filePrefix, all_data_by_user):
    AlleventsCombination = ['B-Alert Decontaminated EEG', 'B-Alert EEG',   'Eyetracker HTC VIVE Pro Eye','R Analysis GazeAnalysis I-VT filter','Shimmer GSR 4B59', 'Shimmer shim exg 02 5F2F ECG']
    i = 0
    print("id,Model#", end=",")

    for item in AlleventsCombination:
        print(item, end=",")
    

    for userId in all_data_by_user.keys():
        print(userId, end=",")
    print("")
    
    for L in range(1,len(AlleventsCombination)+1):
        for subset in itertools.combinations(AlleventsCombination, L):
            eventsCombination = list(subset)
            #print(eventsCombination, " : ", key)
            print(str(i)+",Model "+str(i), end=",")
            for item in AlleventsCombination:
                print(item in eventsCombination, end =",")

            #with open('/home/deep01/zaher/MLTMoutput_2C/MLTM_zaher_model_'+str(i)+'.txt', 'r') as f:
            #with open('/home/deep01/zaher/MLTMoutput_2C/MLTM_zaher_model_'+str(i)+'.txt', 'r') as f:
            for userId in all_data_by_user.keys():
                with open('/home/deep01/zaher/MLTMoutput/by_users/'+userId+str(filePrefix)+str(i)+'.txt', 'r') as f:
                    lines = f.read().splitlines()
                    last_line = lines[-1]
                    #print (last_line)
                    if 'Final Accuracy' in last_line:
                        print(last_line[18:], end =",")
                    else:
                        print('', end =",")
            print('');

            i +=1

## Step 1 - read pickles that contains data exported before classification

### Example 1 - Read training and testing data of any fold for all features group combination (article 2)

In [3]:
all_training_by_event_combination_without_overlap = pd.read_pickle("/home/deep01/zaher/MLSTM-FCN-master/output/all_training_by_event_combination_by_features_overlap_fold1.array")
all_test_by_event_combination_without_overlap = pd.read_pickle("/home/deep01/zaher/MLSTM-FCN-master/output/all_test_by_event_combination_by_features_overlap_fold1.array")


## Step 2 - run printAllResTRAINTestFeaturesWithFolds function in order to read all results from files exported by deep learning architecture

### Example 1 - printing results of all features' groups in all folds (article 2)
#### N.B: You can copy the output then paste it in excel

In [4]:
filePrefix = "MLTM_zaher_2C_no_aug_by_post_all_by_features_"
printAllResTRAINTestFeaturesWithFolds(filePrefix, all_training_by_event_combination_without_overlap, all_test_by_event_combination_without_overlap, 5)

id,Model#,train data #,test data #,Featurs #,eeg_current_columns1,eeg_current_columns2,eeg_current_columns3,et_columns1,et_columns2,et_columns3,ecg_columns1,ecg_columns2,ecg_columns3,fold0,fold1,fold2,fold3,fold4,accuracy
0,Model 0,248,67,6,True,True,False,False,False,False,False,False,False,0.6140350877192983,0.5614035087719298,0.5964912280701754,0.6274509803921569,0.5660377358490566,0.5930837081605234
1,Model 1,249,67,6,True,False,True,False,False,False,False,False,False,0.5964912280701754,0.543859649122807,0.5517241379310345,0.6274509803921569,0.6415094339622641,0.5922070858956876
2,Model 2,242,66,26,True,False,False,True,False,False,False,False,False,0.6,0.5714285714285714,0.6785714285714286,0.66,0.6037735849056604,0.622754716981132
3,Model 3,242,66,5,True,False,False,False,True,False,False,False,False,0.6727272727272727,0.5178571428571429,0.625,0.64,0.6037735849056604,0.6118716000980152
4,Model 4,242,66,7,True,False,False,False,False,True,False,False,False,0.6545454545454545,0.517

59,Model 59,203,51,11,True,False,False,False,False,True,False,True,False,0.3877551020408163,0.5121951219512195,0.3333333333333333,0.6829268292682927,0.4318181818181818,0.4696057136823687
60,Model 60,203,51,9,True,False,False,False,False,True,False,False,True,0.6530612244897959,0.5121951219512195,0.5,0.3902439024390244,0.5681818181818182,0.5247364134123715
61,Model 61,209,52,13,True,False,False,False,False,False,True,True,False,0.6666666666666666,0.6904761904761905,0.6046511627906976,0.7380952380952381,0.6818181818181818,0.6763414879693949
62,Model 62,209,52,11,True,False,False,False,False,False,True,False,True,0.7254901960784313,0.6904761904761905,0.5813953488372093,0.7142857142857143,0.6818181818181818,0.6786931262991455
63,Model 63,209,52,9,True,False,False,False,False,False,False,True,True,0.39215686274509803,0.2619047619047619,0.3488372093023256,0.6428571428571429,0.6818181818181818,0.465514831725502
64,Model 64,242,66,29,False,True,True,True,False,False,False,False,False,0.4727272

109,Model 109,228,57,29,False,False,False,True,False,False,False,True,True,0.45454545454545453,0.5106382978723404,0.5531914893617021,0.3333333333333333,0.41304347826086957,0.45295041067473996
110,Model 110,228,57,12,False,False,False,False,True,True,True,False,False,0.6727272727272727,0.7021276595744681,0.6170212765957447,0.7291666666666666,0.6521739130434783,0.6746433577215261
111,Model 111,228,57,10,False,False,False,False,True,True,False,True,False,0.4,0.5531914893617021,0.3829787234042553,0.3333333333333333,0.41304347826086957,0.41650940487203203
112,Model 112,228,57,8,False,False,False,False,True,True,False,False,True,0.6,0.574468085106383,0.574468085106383,0.4583333333333333,0.5869565217391305,0.5588452050570459
113,Model 113,228,57,12,False,False,False,False,True,False,True,True,False,0.6,0.7446808510638298,0.8085106382978723,0.7291666666666666,0.6086956521739131,0.6982107616404563
114,Model 114,228,57,10,False,False,False,False,True,False,True,False,True,0.6363636363636364,0.68

163,Model 163,203,51,36,True,False,False,True,False,False,True,True,False,0.7959183673469388,0.7073170731707317,0.7142857142857143,0.7560975609756098,0.7272727272727273,0.7401782886103444
164,Model 164,203,51,34,True,False,False,True,False,False,True,False,True,0.7142857142857143,0.6585365853658537,0.7142857142857143,0.6097560975609756,0.7272727272727273,0.684827367754197
165,Model 165,203,51,32,True,False,False,True,False,False,False,True,True,0.5102040816326531,0.4878048780487805,0.8333333333333334,0.5121951219512195,0.5,0.5687074829931973
166,Model 166,203,51,15,True,False,False,False,True,True,True,False,False,0.6938775510204082,0.7073170731707317,0.6190476190476191,0.6341463414634146,0.6590909090909091,0.6626958987586166
167,Model 167,203,51,13,True,False,False,False,True,True,False,True,False,0.3877551020408163,0.5121951219512195,0.35714285714285715,0.6341463414634146,0.4318181818181818,0.4646115208832978
168,Model 168,203,51,11,True,False,False,False,True,True,False,False,True,0

215,Model 215,203,51,36,False,False,True,True,False,True,True,False,False,0.7551020408163265,0.7073170731707317,0.7380952380952381,0.6585365853658537,0.7045454545454546,0.712719278398721
216,Model 216,203,51,34,False,False,True,True,False,True,False,True,False,0.4897959183673469,0.5609756097560976,0.5714285714285714,0.7317073170731707,0.5,0.5707814833250373
217,Model 217,203,51,32,False,False,True,True,False,True,False,False,True,0.5918367346938775,0.5853658536585366,0.5952380952380952,0.34146341463414637,0.5,0.5227808196449312
218,Model 218,203,51,36,False,False,True,True,False,False,True,True,False,0.7755102040816326,0.7073170731707317,0.7380952380952381,0.6097560975609756,0.6136363636363636,0.6888629953089884
219,Model 219,203,51,34,False,False,True,True,False,False,True,False,True,0.7346938775510204,0.7317073170731707,0.6904761904761905,0.6829268292682927,0.7045454545454546,0.7088699337828258
220,Model 220,203,51,32,False,False,True,True,False,False,False,True,True,0.44897959183673

268,Model 268,203,51,39,True,True,False,True,False,False,True,True,False,0.7551020408163265,0.6829268292682927,0.7142857142857143,0.7560975609756098,0.7272727272727273,0.7271369745237342
269,Model 269,203,51,37,True,True,False,True,False,False,True,False,True,0.7755102040816326,0.7317073170731707,0.7142857142857143,0.7317073170731707,0.6818181818181818,0.727005746866374
270,Model 270,203,51,35,True,True,False,True,False,False,False,True,True,0.4897959183673469,0.4634146341463415,0.7142857142857143,0.3902439024390244,0.6590909090909091,0.5433662156658673
271,Model 271,203,51,18,True,True,False,False,True,True,True,False,False,0.7551020408163265,0.7073170731707317,0.5952380952380952,0.5121951219512195,0.6590909090909091,0.6457886480534565
272,Model 272,203,51,16,True,True,False,False,True,True,False,True,False,0.4489795918367347,0.7317073170731707,0.3333333333333333,0.6585365853658537,0.4318181818181818,0.5208750018854549
273,Model 273,203,51,14,True,True,False,False,True,True,False,Fals

318,Model 318,203,51,35,False,True,True,True,True,False,False,True,False,0.5102040816326531,0.5609756097560976,0.6190476190476191,0.6341463414634146,0.5,0.5648747303799568
319,Model 319,203,51,33,False,True,True,True,True,False,False,False,True,0.3469387755102041,0.5609756097560976,0.8333333333333334,0.4146341463414634,0.5,0.5311763729882196
320,Model 320,203,51,39,False,True,True,True,False,True,True,False,False,0.7755102040816326,0.7073170731707317,0.7857142857142857,0.7073170731707317,0.6363636363636364,0.7224444545002036
321,Model 321,203,51,37,False,True,True,True,False,True,False,True,False,0.6326530612244898,0.5609756097560976,0.5952380952380952,0.6585365853658537,0.6136363636363636,0.61220794304418
322,Model 322,203,51,35,False,True,True,True,False,True,False,False,True,0.5102040816326531,0.5609756097560976,0.6428571428571429,0.4146341463414634,0.7272727272727273,0.5711887415720168
323,Model 323,203,51,39,False,True,True,True,False,False,True,True,False,0.7755102040816326,0.658

373,Model 373,203,51,40,True,True,True,True,True,False,True,False,False,0.7346938775510204,0.7073170731707317,0.7380952380952381,0.7804878048780488,0.6363636363636364,0.7193915260117352
374,Model 374,203,51,38,True,True,True,True,True,False,False,True,False,0.4897959183673469,0.43902439024390244,0.6428571428571429,0.5609756097560976,0.7045454545454546,0.5674397031539888
375,Model 375,203,51,36,True,True,True,True,True,False,False,False,True,0.46938775510204084,0.43902439024390244,0.5476190476190477,0.6097560975609756,0.5454545454545454,0.5222483671961025
376,Model 376,203,51,42,True,True,True,True,False,True,True,False,False,0.7755102040816326,0.7073170731707317,0.7380952380952381,0.7317073170731707,0.75,0.7405259664841546
377,Model 377,203,51,40,True,True,True,True,False,True,False,True,False,0.7142857142857143,0.5365853658536586,0.6904761904761905,0.4634146341463415,0.5,0.580952380952381
378,Model 378,203,51,38,True,True,True,True,False,True,False,False,True,0.4489795918367347,0.6097

423,Model 423,203,51,40,True,False,False,True,True,True,True,False,True,0.7551020408163265,0.7560975609756098,0.7380952380952381,0.6097560975609756,0.6136363636363636,0.6945374602169028
424,Model 424,203,51,38,True,False,False,True,True,True,False,True,True,0.5918367346938775,0.5365853658536586,0.8095238095238095,0.36585365853658536,0.5,0.5607599137215862
425,Model 425,203,51,40,True,False,False,True,True,False,True,True,True,0.7755102040816326,0.7317073170731707,0.6428571428571429,0.7804878048780488,0.6818181818181818,0.7224761301416354
426,Model 426,203,51,42,True,False,False,True,False,True,True,True,True,0.7346938775510204,0.6585365853658537,0.7380952380952381,0.7804878048780488,0.6818181818181818,0.7187263375416686
427,Model 427,203,51,21,True,False,False,False,True,True,True,True,True,0.7346938775510204,0.6829268292682927,0.5952380952380952,0.5853658536585366,0.7045454545454546,0.66055402205228
428,Model 428,203,51,41,False,True,True,True,True,True,True,False,False,0.775510204081

468,Model 468,203,51,21,True,True,True,False,True,True,False,True,True,0.4489795918367347,0.6585365853658537,0.35714285714285715,0.6585365853658537,0.4318181818181818,0.5110027603058962
469,Model 469,203,51,23,True,True,True,False,True,False,True,True,True,0.7551020408163265,0.6829268292682927,0.6428571428571429,0.6097560975609756,0.7954545454545454,0.6972193311914566
470,Model 470,203,51,25,True,True,True,False,False,True,True,True,True,0.7959183673469388,0.6829268292682927,0.6428571428571429,0.6829268292682927,0.6590909090909091,0.6927440155663153
471,Model 471,203,51,45,True,True,False,True,True,True,True,True,False,0.8163265306122449,0.7073170731707317,0.7142857142857143,0.7317073170731707,0.6363636363636364,0.7212000543010995
472,Model 472,203,51,43,True,True,False,True,True,True,True,False,True,0.7551020408163265,0.6585365853658537,0.6904761904761905,0.7073170731707317,0.5909090909090909,0.6804681961476385
473,Model 473,203,51,41,True,True,False,True,True,True,False,True,True,0.6

## Another example to print results for all events combination for 5 folds (article 1)

In [5]:
all_training_by_event_combination_overlap = pd.read_pickle("/home/deep01/zaher/MLSTM-FCN-master/output/all_training_by_event_combination_overlap_fold0.array")
all_test_by_event_combination_overlap = pd.read_pickle("/home/deep01/zaher/MLSTM-FCN-master/output/all_test_by_event_combination_overlap_fold0.array")


In [6]:
filePrefix = "MLTM_zaher_2C_with_aug_by_post_"
printAllResTRAINTestWithFolds(filePrefix, all_training_by_event_combination_overlap, all_test_by_event_combination_overlap, 5)

id,Model#,train data #, test data #,Featurs #,B-Alert Decontaminated EEG,B-Alert EEG,Eyetracker HTC VIVE Pro Eye,R Analysis GazeAnalysis I-VT filter,Shimmer GSR 4B59,Shimmer shim exg 02 5F2F ECG,fold0,fold1,fold2,fold3,fold4,accuracy
0,Model 0,3993,959,9,True,False,False,False,False,False,0.6164383561643836,0.6546546546546547,0.641438032166509,0.5864406779661017,0.6132542037586548,0.6224451849420607
1,Model 1,4235,1027,9,False,True,False,False,False,False,0.6214355948869223,0.653393665158371,0.624655013799448,0.6486486486486487,0.6142578125,0.632478146998678
2,Model 2,4901,1253,31,False,False,True,False,False,False,0.5349959774738536,0.6721177432542927,0.6822066822066822,0.6801948051948052,0.47310126582278483,0.6085232947904837
3,Model 3,4874,1253,25,False,False,False,True,False,False,0.6138374899436846,0.5952575633687653,0.5571095571095571,0.5286415711947627,0.5850722311396469,0.5759836825512834
4,Model 4,4790,1152,8,False,False,False,False,True,False,0.5665499124343257,0.615810276679

48,Model 48,3176,781,77,True,False,True,True,False,True,0.7055771725032426,0.7683073229291717,0.7090464547677262,0.6491935483870968,0.7181372549019608,0.7100523506978397
49,Model 49,3190,778,60,True,False,True,False,True,True,0.6614583333333334,0.8607442977190877,0.671604938271605,0.8239247311827957,0.6755126658624849,0.7386489932738614
50,Model 50,3168,778,54,True,False,False,True,True,True,0.7369791666666666,0.6626650660264105,0.6641975308641975,0.6549391069012178,0.6621454993834772,0.676185273968394
51,Model 51,3888,942,73,False,True,True,True,True,False,0.5815450643776824,0.6072507552870091,0.6097804391217565,0.5183867141162515,0.5960912052117264,0.5826108356228852
52,Model 52,3218,795,77,False,True,True,True,False,True,0.710828025477707,0.7909738717339667,0.6420798065296252,0.7272727272727273,0.7156862745098039,0.717368141104766
53,Model 53,3237,792,60,False,True,True,False,True,True,0.7161125319693095,0.833729216152019,0.7863247863247863,0.8348387096774194,0.7478890229191797,0.78